In [5]:
import cv2
import face_recognition
import pandas as pd
from datetime import datetime, timedelta
import os

# Function to mark attendance
def mark_attendance(name):
    current_time = datetime.now()
    # If CSV doesn't exist, create it
    if not os.path.exists('attendance.csv'):
        df = pd.DataFrame({'Name': [name], 'Time': [current_time.strftime('%H:%M:%S')], 'Date': [current_time.strftime('%Y-%m-%d')]})
        df.to_csv('attendance.csv', index=False)
    else:
        # Read existing attendance data
        df = pd.read_csv('attendance.csv')
        
        # Check if name already exists and if it's been an hour since last attendance
        if name in df['Name'].values:
            last_time = pd.to_datetime(df[df['Name'] == name]['Time'].values[0])
            if current_time - last_time < timedelta(hours=0.005):
                print(f"{name}, your attendance has already been marked!")
                return  # Skip marking attendance again

        # If it's not in the list or 1 hour has passed, mark attendance
        new_entry = pd.DataFrame({'Name': [name], 'Time': [current_time.strftime('%H:%M:%S')], 'Date': [current_time.strftime('%Y-%m-%d')]})
        df = pd.concat([df, new_entry], ignore_index=True)
        df.to_csv('attendance.csv', index=False)
        print(f"Attendance marked for {name} at {current_time.strftime('%H:%M:%S')}")

# Function to recognize faces and mark attendance
def recognize_faces():
    cap = cv2.VideoCapture(0)

    known_faces = []  # List to store known face encodings
    known_names = []  # List to store known names
    known_faces, known_names = load_known_faces()  # Load the known faces from a folder
    marked_people = set()  # Set to store people whose attendance is marked

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_encoding, face_location in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_faces, face_encoding)
            name = "Unknown"

            if True in matches:
                match_index = matches.index(True)
                name = known_names[match_index]

                # Mark attendance if face recognized and it's time for new attendance
                if name not in marked_people:
                    mark_attendance(name)
                    marked_people.add(name)
                    cv2.putText(frame, f"Attendance marked for {name}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Draw rectangle around face
            top, right, bottom, left = face_location
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("Face Recognition Attendance System", frame)

        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting...")
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to load known faces from a folder
def load_known_faces(image_path='images'):
    known_faces = []
    known_names = []

    for name in os.listdir(image_path):
        person_folder = os.path.join(image_path, name)
        
        # Ensure it's a directory
        if os.path.isdir(person_folder):
            for filename in os.listdir(person_folder):
                filepath = os.path.join(person_folder, filename)
                
                # Ensure it's a valid file (not a directory)
                if os.path.isfile(filepath):
                    try:
                        # Load the image and compute face encodings
                        image = face_recognition.load_image_file(filepath)
                        encodings = face_recognition.face_encodings(image)
                        if len(encodings) > 0:
                            encoding = encodings[0]
                            known_faces.append(encoding)
                            known_names.append(name)
                    except Exception as e:
                        print(f"Error processing file {filepath}: {e}")
    
    return known_faces, known_names

# Start face recognition
recognize_faces()


Attendance marked for sohel at 23:23:58
Exiting...
